# Raster polygon containment figure

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms
import matplotlib.colors
import rasterio.features
import rasterio.transform
import shapely.affinity
import math
import geopandas as gp


In [ ]:
plt.style.use('asu-light')

In [ ]:
parcel = gp.read_postgis('SELECT gid, apn, ST_Transform(geog::geometry, 26911) as geom FROM diss.buildable_areas WHERE apn = \'433231018\'', 'postgres://matthewc@localhost:5432/matthewc').geometry.iloc[0]

In [ ]:
parcel

In [ ]:
parcel = shapely.affinity.rotate(parcel, 30) # fit both ways in figure

In [ ]:
def rasterize (geom, scalar = 1):
    w, s, e, n = rasterio.features.bounds(geom)
    # force width/height to exact meters so pixels are whole meters
    e = w + math.ceil(e - w)
    n = s + math.ceil(n - s)
    width = int(round(e - w)) * scalar
    height = int(round(n - s)) * scalar

    if width == 0 or height == 0:
        return np.array([False for d in dims])

    xform = rasterio.transform.from_bounds(w, s, e, n, width, height)
    return rasterio.features.geometry_mask(geom, (height, width), xform, invert=True), xform

In [ ]:
def fit_pixel (mask, width, height):
    '''
    This function checks whether a rectangle of width x height can fit inside the rasterized polygon described by
    mask.
    '''
    for x in range(mask.shape[0]):
        for y in range(mask.shape[1]):
            if mask[x, y]:
                if ((x + width) < mask.shape[0] and (y + height) < mask.shape[1]
                        and np.all(mask[x:x + width, y:y + height])):
                    return x, y
    return -1, -1

In [ ]:
cmap = matplotlib.colors.ListedColormap(np.array([[1, 1, 1], [140 / 255, 29 / 255, 64 / 255]]))
graycmap = matplotlib.colors.ListedColormap(np.array([[1, 1, 1], [0.5, 0.5, 0.5]]))
overcmap = matplotlib.colors.ListedColormap(np.array([[1, 1, 1, 0], [1, 198 / 255, 39 / 255, 1]]))

f, axs = plt.subplots(6, 3, figsize=(10, 10), frameon=False)

alpha = 'abcdefghijklmnopqrstuvwxyzαβγδεζηθλμ'

for dim, dim_label, col in (
        ((12, 10), 'SFH/duplex', 1),
        ((16, 11), 'Threeplex', 3),
        ((21, 10), 'Sixplex', 5)
    ):
    for isrot in [False, True]:
        if isrot:
            col += 1
        for row, rot in enumerate(range(0, 90, 15), 1):
            ix = (row - 1) * 6 + col
            plt.subplot(6, 6, ix, frameon=False)

            if row == 1:
                plt.title(f'{dim_label}\nBuilding rotation\n{90 if isrot else 0} degrees')
            if col == 1:
                plt.ylabel(f'Parcel rotation\n{rot} degrees')

            plt.xlabel(f'({alpha[ix - 1]})')

            g = shapely.affinity.rotate(parcel, rot, use_radians=False)
            mask, xform = rasterize(g)

            # find a pixel where the building fits
            if isrot:
                x, y = fit_pixel(mask, *(dim[::-1]))
            else:
                x, y = fit_pixel(mask, *dim)
            overlay = np.zeros_like(mask)
            if x != -1:
                plt.imshow(mask, cmap=cmap, interpolation='nearest')
                if isrot:
                    overlay[x:x + dim[1], y:y + dim[0]] = True
                else:
                    overlay[x:x + dim[0], y:y + dim[1]] = True
                fit = True
            else:
                fit = False
                plt.imshow(mask, cmap=graycmap, interpolation='nearest')

            plt.imshow(overlay, cmap=overcmap, interpolation='nearest')
            plt.xlim(0, 30)
            plt.ylim(0, 30)
            plt.xticks([])
            plt.yticks([])
    
plt.savefig('../../dissertation/fig/construction/polyinpoly.pdf', bbox_inches='tight')

In [ ]:
rnge = range(0, 360, 15)
dim = (100, 120)
for ix, rot in enumerate(rnge, 1):
    plt.subplot(4, len(rnge) // 4, ix, frameon=False)

    g = shapely.affinity.rotate(parcel, rot, use_radians=False)
    mask, xform = rasterize(g, scalar=10)
    
    # to avoid confusion, rotate axes so the building appears to be rotating instead of the parcel
    base = plt.gca().transData
    affine = matplotlib.transforms.Affine2D().rotate_deg_around(150, 150, rot)

    # find a pixel where the building fits
    x, y = fit_pixel(mask, *dim)
    overlay = np.zeros_like(mask)
    if x != -1:
        plt.imshow(mask, cmap=cmap, interpolation='nearest', transform=affine + base)
        overlay[x:x + dim[0], y:y + dim[1]] = True
        fit = True
    else:
        fit = False
        plt.imshow(mask, cmap=graycmap, interpolation='nearest')

    plt.imshow(overlay, cmap=overcmap, interpolation='nearest', transform=affine + base)
    plt.xlim([-25, 325])
    plt.ylim([-25, 325])
    plt.xticks([])
    plt.yticks([])

plt.savefig('../../defense/rotation.pdf', bbox_inches='tight')

In [ ]:
mask.shape